# Demonstration for the upstream regulation network reconstruction

This notebook demonstrates how to use BRAvo to reconstruct a regulation network with specific options and use it as a model, here to infer new knownledge from partial knowledge on nodes evolution.

In [3]:
# Imports
import time
import csv, sys
import logging
from IPython.display import display, Markdown, Latex
import networkx as nx
parentdir = "../src"
sys.path.insert(0, parentdir)
import bravo.regulation as regulation
import bravo.signaling as signaling
import bravo.config as config
import bravo.util as util
import pyBravo as bravo_main

## Network reconstruction

### Global options

We define below the global options regarding the execution:

- Max depth of 10 levels
- Fast reconstruction (look only at standard names)
- Decompose complexes
- Extend search by considering synonyms and removing suffixes
- Include unsinged interactions
- Use Pathway Commons' SPARQL endpoint
- Consider all data sources except MiRTarBase and MSigDB

In [4]:
# Global options

config.MAX_DEPTH = 10
config.HAS_MAX_DEPTH = True
config.FAST = False
config.DECOMPOSE_COMPLEXES = True
config.EXTEND_WITH_SYNONYMS = True
config.EXTEND_WITH_SUFFIXES = True
config.UNKNOWN = True
config.SPARQL_ENDPOINT = "http://rdf.pathwaycommons.org/sparql/"
config.VERBOSE = True

""" all possible data sources """
ds = ['bind', 'biogrid', 'corum',
      'ctd', 'dip', 'drugbank', 'hprd', 'humancyc', 'inoh',
      'intact', 'kegg', 'mirtarbase', 'netpath', 'panther',
      'pid', 'psp', 'reactome', 'reconx', 'smpdb', 'wp',
      'intact_complex', 'msigdb', 'reach', 'wikipathways']

config.DATA_SOURCES = set(ds) - set(['mirtarbase', 'msigdb'])

pybravo_stdout = [''] * 3

### Gene names input list

Here we read the list of input gene names from a file called `input-910.csv`. This file should contain one gene name per line.

In [5]:
# Read input genes list from a file
def read_input_genes(filename):
    res = []
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in reader:
            res.append(''.join(row))
    return res

# Genes input list
gene_list = read_input_genes('data/input-910.csv')
print('{} genes in input list'.format(len(gene_list)))

910 genes in input list


### Initial reconstruction

We now call the reconstruction on the previous list of gene names.
This iterative step is performed by querying the Pathway Commons endpoint and using the results to complete the graph and prepare the next query.
It may take some time, typically between 20 and 30 minutes.

In [6]:
# Add logging handler to file for later analysis
handler = logging.FileHandler('std.log', mode = 'w')
handler.setFormatter(logging.Formatter('%(message)s'))
logging.getLogger().addHandler(handler)

In [7]:
%%capture --no-display pybravo_stdout_0

start_time = time.time()

# Call reconstruction
reconstructed_network = regulation.upstream_regulation(gene_list)

elapsed_time = round((time.time() - start_time), 2)

2020-06-23 12:09:04,208 - INFO: to be explored after complex decomposition ['LUM', 'CPZ', 'DCN', 'ISLR', 'FNDC1', 'THBS2', 'COL16A1', 'COL1A1', 'COL3A1', 'COL5A1', 'SSC5D', 'MMP28', 'GEM', 'DACT3', 'MRC2', 'GPR124', 'LOXL1', 'ST6GAL2', 'COL1A2', 'PRICKLE1', 'LTBP2', 'COL6A3', 'CDR2L', 'PMP22', 'EMILIN1', 'COL14A1', 'CDH11', 'C1QTNF2', 'CCDC80', 'EFEMP1', 'GLT8D2', 'GGT5', 'ANTXR1', 'SNAI1', 'TSHZ3', 'PPAPDC1A', 'ALDH1A3', 'HEPH', 'FBN1', 'ECM1', 'DPYSL3', 'SRPX', 'PODN', 'GFPT2', 'FOXL1', 'COL10A1', 'GAS1', 'ITGA11', 'GLI2', 'HAND2', 'VCAN', 'GPX8', 'POSTN', 'CRISPLD2', 'PTGIR', 'CALHM2', 'FOXF1', 'MOXD1', 'MGP', 'ASPN', 'COL12A1', 'ZNF469', 'CHST3', 'NHS', 'LAMA2', 'AEBP1', 'PCDH7', 'HTRA3', 'HDGFRP3', 'MXRA5', 'HEYL', 'SVEP1', 'MAP1A', 'SPEG', 'GLI3', 'COL6A2', 'F2R', 'FAP', 'C13orf33', 'SEMA3C', 'ALDH1L2', 'LXN', 'PHLDB1', 'PRELP', 'ADAMTS2', 'PMEPA1', 'EPHA3', 'FLNA', 'RSPO3', 'TGFB3', 'TMEM132E', 'SGCA', 'C1QTNF7', 'COL8A2', 'ADAMTS12', 'MFAP4', 'LAMA1', 'PLAT', 'CTBP2', 'COL4A2',

2020-06-23 12:09:04,215 - INFO: new synonmys to be explored:['LDC', 'SLRR2D', 'CSCD', 'DSPG2', 'PG40', 'PGII', 'PGS2', 'SLRR1B', 'HsT17563', 'Meflin', 'AGS8', 'FNDC2', 'MEL4B3', 'bA243O10.1', 'dJ322A24.1', 'TSP2', '447AA', 'FP1572', 'EDSARTH1', 'EDSC', 'OI1', 'OI2', 'OI3', 'OI4', 'EDS4A', 'EDSVASC', 'EDSC', 'EDSCL1', 'S5D-SRCRB', 'EPILYSIN', 'MM28', 'MMP-25', 'MMP-28', 'MMP25', 'KIR', 'DAPPER3', 'RRR1', 'CD280', 'CLEC13E', 'ENDO180', 'UPARAP', 'ADGRA2', 'TEM5', 'LOL', 'LOXL', 'SIAT2', 'ST6GalII', 'EDSARTH2', 'EDSCV', 'OI4', 'EPM1B', 'RILP', 'LTBP3', 'DASS', 'GPHYSD3', 'LTBP-3', 'STHAG6', 'pp6425', 'BTHLM1', 'DYT27', 'UCMD1', 'HUMPPA', 'PXMP2', 'EMI', 'EMILIN', 'gp115', 'UND', 'CAD11', 'CDHOB', 'ESWS', 'OB', 'OSF-4', 'CTRP2', 'zacrp2', 'DRO1', 'SSG1', 'URB', 'okuribin', 'DHRD', 'DRAD', 'FBLN3', 'FBNL', 'FIBL-3', 'MLVT', 'MTLV', 'S1-5', 'GGL', 'GGT 5', 'GGT-REL', 'GGTLA1', 'ATR', 'GAPO', 'TEM8', 'SLUGH2', 'SNA', 'SNAH', 'SNAIL', 'SNAIL1', 'dJ710H13.1', 'TSH3', 'ZNF537', 'PLPP4', 'DPPL2',

2020-06-23 12:10:10,320 - INFO: 		removing suffix  complex for NFX1/SIN3/HDAC complex
2020-06-23 12:10:10,324 - INFO: 				 --> NFX1/SIN3/HDAC
2020-06-23 12:12:39,052 - INFO: 		removing suffix  complex for tungsten carbide/Cobalt complex
2020-06-23 12:12:39,054 - INFO: 				 --> tungsten carbide/Cobalt
2020-06-23 12:12:39,056 - INFO: 		removing suffix  protein for DCN protein
2020-06-23 12:12:39,058 - INFO: 				 --> DCN
2020-06-23 12:12:39,060 - INFO: 		removing suffix  protein for NOG protein
2020-06-23 12:12:39,062 - INFO: 				 --> NOG
2020-06-23 12:12:39,064 - INFO: 		removing suffix  protein for DCN protein
2020-06-23 12:12:39,066 - INFO: 				 --> DCN
2020-06-23 12:12:39,069 - INFO: 		removing suffix  protein for NOG protein
2020-06-23 12:12:39,071 - INFO: 				 --> NOG
2020-06-23 12:12:39,072 - INFO: 		removing suffix  mRNA for LUM mRNA
2020-06-23 12:12:39,073 - INFO: 				 --> LUM
2020-06-23 12:12:39,075 - INFO: 		removing suffix  protein for INS protein
2020-06-23 12:12:39,076 - INF

2020-06-23 12:12:44,685 - INFO: 		removing suffix  mRNA for COL1A1 mRNA
2020-06-23 12:12:44,686 - INFO: 				 --> COL1A1
2020-06-23 12:12:44,687 - INFO: 		removing suffix  protein for IL6 protein
2020-06-23 12:12:44,688 - INFO: 				 --> IL6
2020-06-23 12:12:44,688 - INFO: 		removing suffix  mRNA for COL1A1 mRNA
2020-06-23 12:12:44,689 - INFO: 				 --> COL1A1
2020-06-23 12:12:44,690 - INFO: 		removing suffix  protein for DEFA1 protein
2020-06-23 12:12:44,691 - INFO: 				 --> DEFA1
2020-06-23 12:12:44,692 - INFO: 		removing suffix  mRNA for COL1A1 mRNA
2020-06-23 12:12:44,693 - INFO: 				 --> COL1A1
2020-06-23 12:12:44,694 - INFO: 		removing suffix  protein for LEP protein
2020-06-23 12:12:44,694 - INFO: 				 --> LEP
2020-06-23 12:12:44,695 - INFO: 		removing suffix  mRNA for COL1A1 mRNA
2020-06-23 12:12:44,695 - INFO: 				 --> COL1A1
2020-06-23 12:12:44,697 - INFO: 		removing suffix  protein for ABL1 protein
2020-06-23 12:12:44,698 - INFO: 				 --> ABL1
2020-06-23 12:12:44,698 - INFO: 		

2020-06-23 12:12:52,949 - INFO: 		removing suffix  mRNA for PMP22 mRNA
2020-06-23 12:12:52,950 - INFO: 				 --> PMP22
2020-06-23 12:12:52,951 - INFO: 		removing suffix  protein for NOG protein
2020-06-23 12:12:52,953 - INFO: 				 --> NOG
2020-06-23 12:12:52,954 - INFO: 		removing suffix  mRNA for COL6A3 mRNA
2020-06-23 12:12:52,955 - INFO: 				 --> COL6A3
2020-06-23 12:12:52,957 - INFO: 		removing suffix  protein for INS protein
2020-06-23 12:12:52,958 - INFO: 				 --> INS
2020-06-23 12:12:52,959 - INFO: 		removing suffix  mRNA for PMP22 mRNA
2020-06-23 12:12:52,960 - INFO: 				 --> PMP22
2020-06-23 12:12:52,961 - INFO: 		removing suffix  protein for TGFB1 protein
2020-06-23 12:12:52,962 - INFO: 				 --> TGFB1
2020-06-23 12:12:52,963 - INFO: 		removing suffix  mRNA for COL6A3 mRNA
2020-06-23 12:12:52,963 - INFO: 				 --> COL6A3
2020-06-23 12:12:52,964 - INFO: 		removing suffix  protein for pirinixic acid/PPARA protein complex
2020-06-23 12:12:52,965 - INFO: 				 --> pirinixic acid/PPAR

...

2020-06-23 13:21:55,000 - INFO: 				 --> TP53 gene
2020-06-23 13:21:55,001 - INFO: 		removing suffix  gene for TP53 gene
2020-06-23 13:21:55,002 - INFO: 				 --> TP53
2020-06-23 13:21:55,003 - INFO: 		removing suffix  protein for TP53 protein
2020-06-23 13:21:55,004 - INFO: 				 --> TP53
2020-06-23 13:21:55,007 - INFO: 		removing suffix  protein for PRKN protein
2020-06-23 13:21:55,008 - INFO: 				 --> PRKN
2020-06-23 13:21:55,009 - INFO: 		removing suffix  protein for TP53 protein
2020-06-23 13:21:55,010 - INFO: 				 --> TP53
2020-06-23 13:21:55,013 - INFO: 		removing suffix  protein for NOG protein
2020-06-23 13:21:55,014 - INFO: 				 --> NOG
2020-06-23 13:21:55,017 - INFO: 		removing suffix  mRNA for EGR1 mRNA
2020-06-23 13:21:55,018 - INFO: 				 --> EGR1
2020-06-23 13:21:55,021 - INFO: 		removing suffix  protein for ESR2 protein
2020-06-23 13:21:55,022 - INFO: 				 --> ESR2
2020-06-23 13:21:55,023 - INFO: 		removing suffix  mRNA for EGR1 mRNA
2020-06-23 13:21:55,024 - INFO: 				 --

2020-06-23 13:22:00,636 - INFO: 				 --> NOG
2020-06-23 13:22:00,637 - INFO: 		removing suffix  mRNA for KLF4 mRNA
2020-06-23 13:22:00,638 - INFO: 				 --> KLF4
2020-06-23 13:22:00,641 - INFO: 		removing suffix  protein for NOG protein
2020-06-23 13:22:00,643 - INFO: 				 --> NOG
2020-06-23 13:22:00,645 - INFO: 		removing suffix  protein for PPARG protein
2020-06-23 13:22:00,646 - INFO: 				 --> PPARG
2020-06-23 13:22:00,649 - INFO: 		removing suffix  protein for INS protein
2020-06-23 13:22:00,650 - INFO: 				 --> INS
2020-06-23 13:22:00,651 - INFO: 		removing suffix  protein for PPARG protein
2020-06-23 13:22:00,651 - INFO: 				 --> PPARG
2020-06-23 13:22:00,654 - INFO: 		removing suffix  protein for TGFB1 protein
2020-06-23 13:22:00,657 - INFO: 				 --> TGFB1
2020-06-23 13:22:00,658 - INFO: 		removing suffix  protein for PPARG protein
2020-06-23 13:22:00,659 - INFO: 				 --> PPARG
2020-06-23 13:22:00,660 - INFO: 		removing suffix  protein for IFNG protein
2020-06-23 13:22:00,661 - I

2020-06-23 13:24:37,960 - INFO: 				 --> NOG
2020-06-23 13:24:37,961 - INFO: 		removing suffix  mRNA for MLXIPL mRNA
2020-06-23 13:24:37,963 - INFO: 				 --> MLXIPL
2020-06-23 13:24:37,967 - INFO: 		removing suffix  protein for INS protein
2020-06-23 13:24:37,968 - INFO: 				 --> INS
2020-06-23 13:24:37,969 - INFO: 		removing suffix  mRNA for MLXIPL mRNA
2020-06-23 13:24:37,971 - INFO: 				 --> MLXIPL
2020-06-23 13:24:37,974 - INFO: 		removing suffix  protein for INS protein
2020-06-23 13:24:37,975 - INFO: 				 --> INS
2020-06-23 13:24:37,976 - INFO: 		removing suffix  mRNA for MLXIPL mRNA
2020-06-23 13:24:37,977 - INFO: 				 --> MLXIPL
2020-06-23 13:24:37,980 - INFO: 		removing suffix  protein for TGFB1 protein
2020-06-23 13:24:37,982 - INFO: 				 --> TGFB1
2020-06-23 13:24:37,984 - INFO: 		removing suffix  mRNA for MLXIPL mRNA
2020-06-23 13:24:37,985 - INFO: 				 --> MLXIPL
2020-06-23 13:25:02,429 - INFO: 		removing suffix  protein for TNF protein
2020-06-23 13:25:02,431 - INFO: 				

2020-06-23 13:26:56,401 - INFO: 				 --> Estradiol/ESR2 complex
2020-06-23 13:26:56,402 - INFO: 		removing suffix  complex for Estradiol/ESR2 complex
2020-06-23 13:26:56,403 - INFO: 				 --> Estradiol/ESR2
2020-06-23 13:26:56,404 - INFO: 		removing suffix  mRNA for MYC mRNA
2020-06-23 13:26:56,405 - INFO: 				 --> MYC
2020-06-23 13:26:56,408 - INFO: 		removing suffix  mutant form for STK11 mutant form
2020-06-23 13:26:56,409 - INFO: 				 --> STK11
2020-06-23 13:26:56,411 - INFO: 		removing suffix  mRNA for MYC mRNA
2020-06-23 13:26:56,412 - INFO: 				 --> MYC
2020-06-23 13:26:56,417 - INFO: 		removing suffix  protein for ESR1 protein
2020-06-23 13:26:56,419 - INFO: 				 --> ESR1
2020-06-23 13:26:56,420 - INFO: 		removing suffix  mRNA for MYC mRNA
2020-06-23 13:26:56,426 - INFO: 				 --> MYC
2020-06-23 13:26:56,429 - INFO: 		removing suffix  protein for TNF protein
2020-06-23 13:26:56,430 - INFO: 				 --> TNF
2020-06-23 13:26:56,431 - INFO: 		removing suffix  mRNA for MYC mRNA
2020-06-2

2020-06-23 13:27:02,104 - INFO: 				 --> PPARG
2020-06-23 13:27:02,107 - INFO: 		removing suffix  protein for TGFB1 protein
2020-06-23 13:27:02,108 - INFO: 				 --> TGFB1
2020-06-23 13:27:02,109 - INFO: 		removing suffix  protein for PPARG protein
2020-06-23 13:27:02,110 - INFO: 				 --> PPARG
2020-06-23 13:27:02,111 - INFO: 		removing suffix  protein for TGFB1 protein
2020-06-23 13:27:02,112 - INFO: 				 --> TGFB1
2020-06-23 13:27:02,113 - INFO: 		removing suffix  mRNA for AHR mRNA
2020-06-23 13:27:02,114 - INFO: 				 --> AHR
2020-06-23 13:27:02,115 - INFO: 		removing suffix  protein for IFNG protein
2020-06-23 13:27:02,116 - INFO: 				 --> IFNG
2020-06-23 13:27:02,117 - INFO: 		removing suffix  protein for PPARG protein
2020-06-23 13:27:02,118 - INFO: 				 --> PPARG
2020-06-23 13:27:02,121 - INFO: 		removing suffix  protein for IFNG protein
2020-06-23 13:27:02,126 - INFO: 				 --> IFNG
2020-06-23 13:27:02,127 - INFO: 		removing suffix  protein for PPARG protein
2020-06-23 13:27:02,12

In [8]:
print("--- Upstream regulation network reconstructed in {} seconds ---".format(elapsed_time))

--- Upstream regulation network reconstructed in 4720.35 seconds ---


In [9]:
%%capture --no-display pybravo_stdout_1

# Build digraph representation in memory
G = bravo_main.build_nx_digraph(reconstructed_network)

bravo_main.write_to_SIF(G, 'out.sif')
bravo_main.write_provenance(G, 'out-provenance.csv')

### Synonym-based unification

We now perform a synonym-unification, which consists in fusing nodes together if they are synonyms.
This process is useful because of the high heterogeneity of the data sources that compose Pathway Commons.
It typically takes a few seconds.

In [10]:
%%capture --no-display pybravo_stdout_2

start_time = time.time()
# Perform unification
G_prime = util.fast_reg_network_unification(G, util.index_syn)
elapsed_time = round((time.time() - start_time), 2)
print("--- Network simplification in %s seconds ---" % elapsed_time)

# Write to files
bravo_main.write_to_SIF(G_prime, 'out-unified.sif')
bravo_main.write_provenance(G_prime, 'out-unified-provenance.csv')

# Count nodes/edges
print('Nodes after simplification = ' + str(len(G_prime.nodes())))
print('Edges after simplification = ' + str(len(G_prime.edges())))

# Compute centrality
print(bravo_main.get_centrality_as_md(G_prime))

2020-06-23 13:31:59,352 - INFO: 		merging node SNAIL into node SNAI1
2020-06-23 13:31:59,384 - INFO: 		merging node FOXM1B into node FOXM1
2020-06-23 13:31:59,400 - INFO: 		merging node FOXM1B into node FOXM1
2020-06-23 13:31:59,417 - INFO: 		merging node AP1 into node JUN
2020-06-23 13:31:59,430 - INFO: 		merging node AP1 into node JUN
2020-06-23 13:31:59,448 - INFO: 		merging node ATF2 into node GDNF
2020-06-23 13:31:59,462 - INFO: 		merging node ATF1 into node GDNF
2020-06-23 13:31:59,477 - INFO: 		merging node ATF2 into node GDNF
2020-06-23 13:31:59,509 - INFO: 		merging node RAD1 into node RRAD
2020-06-23 13:31:59,601 - INFO: 		merging node JUNB into node RNASEH2A
2020-06-23 13:31:59,616 - INFO: 		merging node IL8 into node CXCL8
2020-06-23 13:31:59,645 - INFO: 		merging node MYC into node PVT1
2020-06-23 13:31:59,669 - INFO: 		merging node SNAIL into node SNAI1
2020-06-23 13:31:59,688 - INFO: 		merging node CSPG2 into node VCAN
2020-06-23 13:31:59,706 - INFO: 		merging node SM22 

2020-06-23 13:32:02,707 - INFO: 		merging node LKB1 into node STK11
2020-06-23 13:32:02,723 - INFO: 		merging node TCF3 into node TCF7L1
2020-06-23 13:32:02,737 - INFO: 		merging node BRG1 into node SMARCA4
2020-06-23 13:32:02,752 - INFO: 		merging node TCF1 into node HNF1A
2020-06-23 13:32:02,765 - INFO: 		merging node TLE5 into node AES
2020-06-23 13:32:02,778 - INFO: 		merging node BRM into node SMARCA2
2020-06-23 13:32:02,794 - INFO: 		merging node TIF1B into node TRIM28
2020-06-23 13:32:02,816 - INFO: 		merging node MIM into node MTSS1
2020-06-23 13:32:02,858 - INFO: 		merging node CD3G into node TARP
2020-06-23 13:32:02,944 - INFO: 		merging node FOXO3A into node FOXO3
2020-06-23 13:32:02,971 - INFO: 		merging node MCG10 into node PCBP4
2020-06-23 13:32:02,983 - INFO: 		merging node MCG10 into node PCBP4
2020-06-23 13:32:02,997 - INFO: 		merging node PGC1A into node PPARGC1A
2020-06-23 13:32:03,011 - INFO: 		merging node MNSOD into node SOD2
2020-06-23 13:32:03,028 - INFO: 		merg

### Saving output

Here we simply save all output provided by pyBRAvo during the reconstruction into a texte file.

In [11]:
all_stdout = '\n'.join([s.stdout for s in [pybravo_stdout_0, pybravo_stdout_1, pybravo_stdout_2]])
print(all_stdout)

Explored 16 regulators
Explored 16 regulators
Explored 26 regulators
Explored 26 regulators
Explored 29 regulators
Explored 33 regulators
Explored 37 regulators
Explored 41 regulators
Explored 43 regulators
Explored 44 regulators
Explored 44 regulators
Explored 51 regulators
Explored 57 regulators
Explored 57 regulators
Explored 57 regulators
Explored 57 regulators
Explored 58 regulators
Explored 59 regulators
Explored 78 regulators
Explored 78 regulators
Explored 80 regulators
Explored 81 regulators
Explored 81 regulators
Explored 81 regulators
Explored 81 regulators
Explored 81 regulators
Explored 81 regulators
Explored 82 regulators
Explored 82 regulators
Explored 82 regulators
Explored 84 regulators
Explored 84 regulators
Explored 84 regulators
Explored 84 regulators
Explored 84 regulators
Explored 87 regulators
Explored 87 regulators
Explored 87 regulators
Explored 87 regulators
Explored 87 regulators
Explored 87 regulators
Explored 91 regulators
Explored 91 regulators
Explored 91

In [12]:
with open('std.out', 'w') as outfile:
    print(all_stdout, file = outfile)

### Recovering logs

Here we read all logs saved to a file during the reconstruction for a later analysis.

In [13]:
with open('std.log', 'r') as logfile:
    pybravo_log_0 = logfile.read()

### Statistics extraction

Here we call several Bash and Python scripts on the results to extracts statistics on the provenance, coverage and content of the network. Some data are summed up in file `cytoscape/edges-provenance.csv` that can be loaded in Cytoscape and applied to the edges of the network.

In [15]:
%%bash

# Extract genes of the unified and non-unified results
sh scripts/get-genes-from-sif.sh out-unified.sif > out-unified.sif.genes
# Provenance of all edges in unified result
bash scripts/pybravo-provenances.sh out-unified-provenance.csv > provenance-unified.txt
# Provenance of signed edges only in unified result
bash scripts/pybravo-provenances.sh <(grep -v 'UNKNOWN' out-unified-provenance.csv) > provenance-unified-signed.txt

# Coverage
python3 scripts/compute_coverage.py out-unified.sif data/input-910.csv Homo_sapiens.gene_info > coverage.txt

# Provenance CSV for Cytoscape
mkdir cytoscape
echo "edge name,provenance" > cytoscape/edges-provenance.csv
sed 's/\t\(.*\)\t/ (\1) /;' out-unified-provenance.csv >> cytoscape/edges-provenance.csv

mkdir: impossible de créer le répertoire «cytoscape»: Le fichier existe


In [16]:
%%bash --no-raise-error
bash scripts/stats-2.sh "out-unified.sif" "std.log"

Nodes: 1630 out-unified.sif.genes
Edges: 4283 out-unified.sif
 - signed: 3715
 - unsigned: 0
 - PART_OF: 568

Coverage: 704 genes are present over 910

Complexes: 299
Complexes involving small molecules: 111


## Using Iggy to make predictions

In this part we use the previously extracted network as a model in order to make predictions on it, using the tool [Iggy](https://github.com/bioasp/iggy).

Using some preliminary observations on some nodes, consisting in signs “$+$” (observed up-regulation) and “$-$” (observed down-regulation), Iggy tries to infer new information on the other nodes which can thus also be predicted as “$+$” or “$-$”. This call relies on Bash and Ptyhon scripts.

For this step, you need Iggy 1.4.1 installed, for instance using pip:

> ```pip install iggy==1.4.1```

Once Iggy is installed, please provide the path to it below.
Please note that this can change according to the installed sofware in your machine.
For instance, the default when only pip installed is `~/.local/bin/iggy.py`.
However, if Conda or Miniconda is installed, the default becomes `~/(path-to-conda-environment)/bin/iggy.py`.

In [17]:
#iggy_command = "${HOME}/.local/bin/iggy.py"
iggy_command = "${CONDA_PREFIX}/bin/iggy.py"

In [18]:
# Interpret Iggy command line
iggy_command = !echo "$iggy_command"
iggy_command = iggy_command[0]
print(iggy_command)

/home/max/miniconda3/envs/pybravo/bin/iggy.py


The scripts below call Iggy, with some data preparation beforehand and a comparison of the result with the original ICGC experimental data afterwards.

The printed output sums up the results of the prediction, and the consistency of these results with the initial ICGC expression data (matching or not matching predictions).

The full result can be found in file `cytoscape/nodes-iggy.csv` in a format that can be fed to Cytoscape in order to label nodes with their observation/prediction sign.

In [19]:
%%bash -s "$iggy_command"
export LC_ALL=C
# Make folders for execution results
mkdir iggy
cd iggy
mkdir result

# Create graph (SIF) file (removes unsigned edges)
bash ../scripts/format-iggy.sh ../out-unified.sif | grep -v 'UNKNOWN' > out-unified-iggy.sif

# Create observations (OBS) file
bash ../scripts/construct-inputs.sh out-unified-iggy.sif ../data/updownreg-v4.3-noinputs.obs > updownreg.obs

# Extract true observations (actually in network)
bash ../scripts/get-genes-from-sif.sh out-unified-iggy.sif > out-unified-iggy.sif.genes 
grep -w -f out-unified-iggy.sif.genes updownreg.obs > updownreg_true.obs

# Run Iggy workflow
bash ../scripts/workflow-iggy.sh --iggy-command $1 updownreg_true.obs out-unified-iggy.sif ../data/icgc.syn.csv 0 0 result/iggy.out > result/out-unified.tsv
cp result/out-unified.tsv ../cytoscape/nodes-iggy.csv

# Output stats
bash ../scripts/stats-result.sh --en --skip0 result/out-unified.tsv out-unified-iggy.sif > result/stats.txt
cat result/stats.txt

Total number of nodes: 1630
Total number of edges: 4283
Number of observations: 654
Number of predictions: 73
  positive (+): 37
  negative (-): 31
  no-change (0): 0
  may-up (NOT-): 0
  may-down (NOT+): 0
  change (CHANGE/NOT0): 5

Predictions found in ICGC data: 55
    +: 26
    -: 27
    0: 0
  coherent: 22
    +: 15
    -: 7
  weakly coherent: 2
    NOT-: 0
    NOT+: 0
    CHANGE: 2
  not coherent: 31
    predicted +: 11
      (should be -)
     predicted -: 20
      (should be +)
     predicted NOT-/NOT+/CHANGE: 0
